In [1]:
import Pkg; Pkg.activate(".")

  Activating project at `c:\Users\anton\.julia\dev\Backboner`

In [5]:
using Revise, Backboner

In [6]:
using Rotations: QuatRotation, params

In [270]:
frames = Frames(
    [
        params(QuatRotation([0 0 1; 1 0 0; 0 1 0])^1);;
        params(QuatRotation([0 0 1; 1 0 0; 0 1 0])^2);;
        params(QuatRotation([0 0 1; 1 0 0; 0 1 0])^3);;
    ],
    [
        0 10 100;
        0 0 0;
        0 0 0;
    ]
)

3-element Frames{Float64}:
 ([0.0 0.0 1.0; 1.0 0.0 0.0; 0.0 1.0 0.0], [0.0, 0.0, 0.0])
 ([0.0 1.0 0.0; 0.0 0.0 1.0; 1.0 0.0 0.0], [10.0, 0.0, 0.0])
 ([1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [100.0, 0.0, 0.0])

In [302]:
standard_coords = [3 1 -4; 1 -1 0; 0 0 0]
backbone = Backbone(frames, standard_coords)

9-element Backbone{Float64, Matrix{Float64}}:
 [0.0, 3.0, 1.0]
 [0.0, 1.0, -1.0]
 [0.0, -4.0, 0.0]
 [11.0, 0.0, 3.0]
 [9.0, 0.0, 1.0]
 [10.0, 0.0, -4.0]
 [103.0, 1.0, 0.0]
 [101.0, -1.0, 0.0]
 [96.0, 0.0, 0.0]

In [303]:
backbone.coords

3×9 Matrix{Float64}:
 0.0   0.0   0.0  11.0  9.0  10.0  103.0  101.0  96.0
 3.0   1.0  -4.0   0.0  0.0   0.0    1.0   -1.0   0.0
 1.0  -1.0   0.0   3.0  1.0  -4.0    0.0    0.0   0.0

In [286]:
Frames(backbone, standard_frame)[1][2]

3-element Vector{Float64}:
 0.0
 0.0
 0.0

In [287]:
frames[1][1] # different because we only have 2 points in the standard frame and rotation is equivalent to 90 degree rotation around z axis

3×3 QuatRotation{Float64} with indices SOneTo(3)×SOneTo(3)(QuaternionF64(0.5, 0.5, 0.5, 0.5)):
 0.0  0.0  1.0
 1.0  0.0  0.0
 0.0  1.0  0.0

In [29]:
backbone.coords == [
    0  0  1  1  1  3;
   -1  1  0  0  0  0;
    0  0 -1  1  0  0;
]

true